In [1]:
# load libraries and dependencies
%pip install xlrd
%pip install openpyxl
import numpy as np
import pickle
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.svm import LinearSVC
#from sklearn import svm 
from sklearn.calibration import CalibratedClassifierCV
from nltk.corpus import stopwords
import time
from nltk.stem import WordNetLemmatizer
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import metrics
from datetime import datetime
from joblib import dump, load
import json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# load train data created from LoadData_TestTrainSplit.ipynb
df_train = pd.read_csv('train.csv')
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
test_all = pd.read_csv('test.csv')
test_all.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
########################################################################################################################
# Process Text (case, punctuation, stopwords, verb tense)
########################################################################################################################
description = df_train.text.tolist()

description = [str(i) for i in description]

description = [i.lower() for i in description]

description = [re.sub(r'[^\w\s]',' ', s) for s in description]

description = [i.split(' ') for i in description]

lemmatizer = WordNetLemmatizer()

sw = stopwords.words('english')

sw.append('')
sw.append('nan')

#description = [[j for j in i if (j not in sw)] for i in description]

print('lemmatizing...')
description = [[lemmatizer.lemmatize(j, pos='v') for j in i] for i in description]

for d in range(len(description)):
    for t in range(len(description[d])):
        if description[d][t].startswith('\n'):
            description[d][t] = description[d][t][1:]
            
description = [[j for j in i if (j not in sw)] for i in description]
df_train['description'] = description

#shuffle train data set so that it trains more randomly instead of over the 2011-2022 timeline
df_train = df_train.sample(frac=1).reset_index(drop=True)
# save this so we can load it more easily in the future
df_train.to_csv('df_train_shuffle_processed.csv')

lemmatizing...


In [8]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.corpora
model = Word2Vec(sentences=description, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
gensim_model = Word2Vec.load("word2vec.model")
# load embeddings generated from Gensim script in separate notebook
myembeddings = {i:j for i,j in zip(gensim_model.wv.index_to_key, range(len(gensim_model.wv.index_to_key)))}
print(len(myembeddings))
myembeddings

19904


{'co': 0,
 'http': 1,
 'get': 2,
 'https': 3,
 'like': 4,
 'fire': 5,
 'û_': 6,
 'amp': 7,
 'go': 8,
 'bomb': 9,
 'new': 10,
 'via': 11,
 '2': 12,
 'people': 13,
 'say': 14,
 'one': 15,
 'news': 16,
 'burn': 17,
 'kill': 18,
 'make': 19,
 'time': 20,
 'flood': 21,
 'video': 22,
 'crash': 23,
 'come': 24,
 'disaster': 25,
 'emergency': 26,
 'build': 27,
 'body': 28,
 'attack': 29,
 'see': 30,
 'look': 31,
 'home': 32,
 'police': 33,
 'take': 34,
 'know': 35,
 'think': 36,
 'would': 37,
 'u': 38,
 'love': 39,
 '3': 40,
 'still': 41,
 'train': 42,
 'storm': 43,
 'back': 44,
 'us': 45,
 'california': 46,
 'bag': 47,
 'suicide': 48,
 'want': 49,
 'collapse': 50,
 'watch': 51,
 'live': 52,
 'day': 53,
 'man': 54,
 'scream': 55,
 '1': 56,
 'work': 57,
 'first': 58,
 'rt': 59,
 'cause': 60,
 'let': 61,
 'world': 62,
 'nuclear': 63,
 'two': 64,
 'need': 65,
 'war': 66,
 'drown': 67,
 'today': 68,
 'û': 69,
 'wreck': 70,
 'youtube': 71,
 'year': 72,
 'dead': 73,
 'plan': 74,
 'destroy': 75,
 '5'

In [9]:
#description = df_train.description
vocab = []
[[vocab.append(j) for j in i] for i in description]
vocab = list(set(vocab))
myembeddings = {w:i for w,i in zip(vocab, range(len(vocab)))}
# saving embeddings here in the data processing stage
# the embeddings depend on the comment order and the way df_train is shuffled

In [10]:
# saving embeddings here in the data processing stage
pd.DataFrame({'word' : list(myembeddings.keys()),
             'index' : list(myembeddings.values())}).to_csv('myembeddings_{}.csv'.format(datetime.now().strftime('%Y%m%d%H%M')), index=False)

In [11]:
# make x_val and y_val
# x_val is the test data narrative and product description
# converted to tokens mapped to the myembeddings
description = test_all.text.tolist()

description = [str(i) for i in description]

description = [i.lower() for i in description]

description = [re.sub(r'[^\w\s]',' ', s) for s in description]

description = [i.split(' ') for i in description]

lemmatizer = WordNetLemmatizer()

sw = stopwords.words('english')

sw.append('')

#description = [[j for j in i if (j not in sw)] for i in description]

print('lemmatizing...')
description = [[lemmatizer.lemmatize(j, pos='v') for j in i] for i in description]

for d in range(len(description)):
    for t in range(len(description[d])):
        if description[d][t].startswith('\n'):
            description[d][t] = description[d][t][1:]
            
description = [[j for j in i if (j not in sw)] for i in description]
test_all['description'] = description
# save this so we can load it more easily in the future
test_all.to_csv('test_all_processed.csv')


lemmatizing...


In [12]:
print('Done.')

Done.
